## Examen 1. Minimizar la función dada

In [1]:
import numpy as np
import pandas as pd
import random

## Función Objetivo

In [2]:
def funcion_objetivo(x):
    # Assuming `x` is a numpy array with two elements, x[0] and x[1]
    sum1 = sum(i * np.cos((i + 1) * x[0] + i) for i in range(1, 6))
    sum2 = sum(i * np.cos((i + 1) * x[1] + i) for i in range(1, 6))
    return sum1 * sum2

## Generar Poblaciónes Iniciales

In [3]:
def inicializar_poblacion(pop_size, bounds, random_state=None):
    if random_state is not None:
        np.random.seed(random_state)
    population = []
    for _ in range(pop_size):
        individual = np.random.uniform(bounds[0], bounds[1], size=2)  # Two genes for x1 and x2
        population.append(individual)
    return np.array(population)

## Evaluar la poblacion

In [4]:
def evaluacion_aptitud(poblacion):
    n = len(poblacion)
    aptitud = []
    matriz_con_aptitud = pd.DataFrame()
    
    for individuo in poblacion:
        aptitud.append(-funcion_objetivo(individuo))  # Evaluamos la aptitud usando la función objetivo
    
    matriz_con_aptitud["POBLACION"] = [list(ind) for ind in poblacion]  # Guardamos cada individuo
    matriz_con_aptitud["APTITUD"] = aptitud  # Aptitud de cada individuo
    
    return matriz_con_aptitud

# Ejemplo de uso

## Metodos de selección

### Ranking

In [5]:
def rank_seleccion(poblacion):
    n = len(poblacion)  # Número de individuos en la población
    poblacion_hijos = []
    i = 0
    
    # Generamos hijos mediante cruce de pares adyacentes en la población ordenada
    while i < n - 1:
        padre1 = poblacion[i]
        padre2 = poblacion[i + 1]
        
        # Realizamos cruce (un punto o homogéneo)
        hijo_1, hijo_2 = cruza_un_corte(padre1, padre2)  # Puedes cambiar a cruza_homogenea si lo prefieres
        
        poblacion_hijos.append(hijo_1)
        poblacion_hijos.append(hijo_2)
        
        i += 2  # Avanzamos de dos en dos para el cruce de pares
    
    return np.array(poblacion_hijos)

### Tournament

In [6]:
def tournament(poblacion):
    n = len(poblacion)  # número de individuos en la población
    num_genes = len(poblacion[0])  # número de genes por individuo
    poblacion_hijos = []

    for i in range(n // 2):
        # Tomamos padres en pares (por ejemplo: 1-10, 2-9, etc., si la población está ordenada)
        padre1 = poblacion[i]
        padre2 = poblacion[n - 1 - i]
        
        # Aplicamos cruce homogéneo
        #hijo_1, hijo_2 = cruza_homogenea(padre1, padre2)
        hijo_1, hijo_2 = cruza_un_corte(padre1, padre2)
        
        # Añadimos los hijos generados a la nueva población de hijos
        poblacion_hijos.append(hijo_1)
        poblacion_hijos.append(hijo_2)
    
    return np.array(poblacion_hijos)


### Aleatorio Monogamico

In [7]:
def aleatorio_monogamico(poblacion):
    n = len(poblacion)  # Número de individuos en la población
    poblacion_disponible = poblacion.copy().tolist()  # Copia de la población para seleccionar y eliminar individuos
    poblacion_hijos = []
    
    # Generamos n/2 pares de padres para obtener una nueva población de hijos
    while len(poblacion_hijos) < n:
        # Selección aleatoria de una pareja de padres
        pareja = random.sample(poblacion_disponible, 2)
        
        # Aplicamos cruce homogéneo entre los padres seleccionados
        hijo_1, hijo_2 = cruza_homogenea(np.array(pareja[0]), np.array(pareja[1]))
        
        # Añadimos los hijos generados a la nueva población de hijos
        poblacion_hijos.append(hijo_1)
        poblacion_hijos.append(hijo_2)
        
        # Eliminamos los padres seleccionados de la población disponible para evitar repetición
        poblacion_disponible.remove(pareja[0])
        poblacion_disponible.remove(pareja[1])
    
    return np.array(poblacion_hijos[:n])  # Retornamos exactamente n hijos

## Metodos de Cruza

### Cruza por un corte

In [8]:
def cruza_un_corte(padre1, padre2):
    num_genes = len(padre1)
    punto_corte = num_genes // 2  # Definimos el punto de corte en la mitad

    # Creamos los hijos usando el cruce en el punto definido
    hijo_1 = np.concatenate((padre1[:punto_corte], padre2[punto_corte:]))
    hijo_2 = np.concatenate((padre2[:punto_corte], padre1[punto_corte:]))
    
    return hijo_1, hijo_2

### Cruza Homogenea

In [9]:
def cruza_homogenea(padre1, padre2):
    num_genes = len(padre1)
    hijo_1 = np.zeros(num_genes)
    hijo_2 = np.zeros(num_genes)
    
    for i in range(num_genes):
        # Seleccionamos aleatoriamente si cada gen viene del padre1 o del padre2
        if random.random() < 0.5:
            hijo_1[i] = padre1[i]
            hijo_2[i] = padre2[i]
        else:
            hijo_1[i] = padre2[i]
            hijo_2[i] = padre1[i]
    
    return hijo_1, hijo_2


In [10]:
pop_size = 20
num_generaciones = 20
mutation_rate = 0.1
elite_size = 2
bounds = [-10, 10]

poblacion1 = inicializar_poblacion(pop_size, bounds, random_state=0)
#poblacion1

In [11]:
matriz_aptitud = evaluacion_aptitud(poblacion1)
#matriz_aptitud

In [12]:
matriz_con_aptitud = matriz_aptitud.sort_values(by="APTITUD", ascending = True).reset_index(drop=True)
#matriz_con_aptitud

In [13]:
poblacion_hijos = tournament(matriz_con_aptitud["POBLACION"])
#poblacion_hijos

In [17]:
def run_GA(poblacion, generaciones,pop_size ):

    print("Comienza el Algoritmo Genetico")
    #Variables para guardar el historico de cada generacion
    mean_fitness_history = []
    best_fitness_history = []
    snapshots = {}
    
    for i in range(generaciones):
        #Guardar valores para comparar entre la primera, mitad y ultima generacion
        if generation == 0:
            snapshots["first"] = population.copy()
        elif generation == num_generations // 2:
            snapshots["middle"] = population.copy()
        elif generation == num_generations - 1:
            snapshots["last"] = population.copy()
    
        #Evaluar a los pobladores
        poblacion_aptitud = evaluacion_aptitud(poblacion)
        matriz_con_aptitud = poblacion_aptitud.sort_values(by="APTITUD", ascending = True).reset_index(drop=True)

        #Obtener la media y la mejor aptitud
        mean_fitness = np.mean(matriz_con_aptitud['APTITUD'])
        best_fitness = np.max(matriz_con_aptitud['APTITUD'])  # Max because we're using negative values for minimization
        
        #Guardar la media y mejor aptitud
        mean_fitness_history.append(mean_fitness)
        best_fitness_history.append(best_fitness)

        poblacion_hijos = rank_seleccion(matriz_con_aptitud["POBLACION"])

        aptitud_hijos = evaluacion_aptitud(poblacion_hijos)

        matriz_mixed = pd.concat([matriz_con_aptitud, aptitud_hijos], ignore_index=True)

        matriz_final = matriz_mixed.sort_values(by="APTITUD", ascending = True).reset_index(drop=True)

        matriz_con_aptitud = matriz_final[0:pop_size]
        
    return matriz_con_aptitud

In [15]:
prueba1 = run_GA(poblacion1,num_generaciones,pop_size)

Comienza el Algoritmo Genetico


In [16]:
prueba1

,POBLACION,APTITUD
0,"[-0.7704127549413631, 5.610583525729108]",-170.301292
1,"[-0.876993355669029, 5.610583525729108]",-160.173845
2,"[-0.7704127549413631, 1.3686789773729693]",-39.952492
3,"[5.563135018997009, 2.3386799374951384]",-38.106746
4,"[-0.876993355669029, 1.3686789773729693]",-37.576604
5,"[-7.634511482621336, 2.798420426550477]",-31.531950
6,"[-1.5269040132219054, 2.798420426550477]",-29.716123
7,"[-7.634511482621336, 2.917882261333123]",-19.649304
8,"[-1.5269040132219054, 2.917882261333123]",-18.517762
9,"[0.43696643500143395, 4.30378732744839]",-18.045946
